In [12]:
# !pip install langchain
# !pip install langchainhub
# !pip install chromadb
# !pip install -U langchain-community
# !pip install gpt4all
# !pip install PyMuPDF
# !pip install llama-cpp-python

In [20]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter  
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.vectorstores.chroma import Chroma
# from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import StrOutputParser
from langchain.docstore.document import Document
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
import os

In [2]:
PATH = r"./Books/"

In [3]:
def read_docs(path):

    if os.path.isdir(path): 
        DATA = []
        for file in os.listdir(path):
            if(file.endswith('.pdf')):
                loader = PyMuPDFLoader(path + file)
                data = loader.load()
                DATA.extend(data)
        return DATA
        
    elif(path.endswith('.pdf')): # for reading single pdf
        loader = PyMuPDFLoader(path)
        data = loader.load()
        return data

In [4]:
docs = read_docs(PATH)
print(len(docs))

1819


In [5]:
print(type(docs))
print(type(docs[100]))

<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [6]:
print(docs[100])
print(docs[100].metadata)
print(docs[100].page_content)

page_content='90     CHAPTER 2  •  APPLICATION LAYER
 
coding techniques to encode digitized voice or video at a rate that matches the cur-
rently available throughput.
While bandwidth-sensitive applications have specific throughput requirements, 
elastic applications can make use of as much, or as little, throughput as happens to 
be available. Electronic mail, file transfer, and Web transfers are all elastic applica-
tions. Of course, the more throughput, the better. There’s an adage that says that one 
cannot be too rich, too thin, or have too much throughput!
Timing
A transport-layer protocol can also provide timing guarantees. As with throughput 
guarantees, timing guarantees can come in many shapes and forms. An example 
guarantee might be that every bit that the sender pumps into the socket arrives 
at the receiver’s socket no more than 100 msec later. Such a service would be 
appealing to interactive real-time applications, such as Internet telephony, virtual 
environments, tel

In [7]:
def make_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,  # Size of each chunk in characters
        chunk_overlap=20,  # Overlap between consecutive chunks
        length_function=len,  # Function to compute the length of the text
        add_start_index=True,  # Flag to add start index to each chunk
    )

    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.\n")
    
    document = chunks[1000]
    print(document.page_content)
    print(document.metadata)

    return chunks 

In [8]:
all_chunks = make_chunks(docs)

Split 1819 documents into 10779 chunks.

R4. For a P2P file-sharing application, do you agree with the statement, “There is no 
notion of client and server sides of a communication session”? Why or why not?
 R5. What information is used by a process running on one host to identify a pro-
cess running on another host?
 R6. Suppose you wanted to do a transaction from a remote client to a server as 
fast as possible. Would you use UDP or TCP? Why?
 R7. Referring to Figure 2.4, we see that none of the applications listed in Figure
{'source': './Books/James W. Kurose, Keith W. Ross - Computer Networking_ A Top-Down Approach-Pearson (2021).pdf', 'file_path': './Books/James W. Kurose, Keith W. Ross - Computer Networking_ A Top-Down Approach-Pearson (2021).pdf', 'page': 176, 'total_pages': 775, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 11.1 (Build 20C69) Quartz PDFContext', 'creationDate': 'D:20210119115037Z', 'modD

In [9]:
for chunk in all_chunks[:10]:
    print(chunk)
    print("\n")

page_content='James F. Kurose
University of Massachusetts, Amherst
Keith W. Ross
Polytechnic Institute of NYU
COMPUTER NETWORKING
A Top-Down Approach' metadata={'source': './Books/James W. Kurose, Keith W. Ross - Computer Networking_ A Top-Down Approach-Pearson (2021).pdf', 'file_path': './Books/James W. Kurose, Keith W. Ross - Computer Networking_ A Top-Down Approach-Pearson (2021).pdf', 'page': 1, 'total_pages': 775, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 11.1 (Build 20C69) Quartz PDFContext', 'creationDate': 'D:20210119115037Z', 'modDate': "D:20210119233105+08'00'", 'trapped': '', 'start_index': 0}


page_content='xix
1
81
181
303
377
449
531
Chapter 1 Computer Networks and the Internet 
Chapter 2 Application Layer  
Chapter 3 Transport Layer  
Chapter 4 The Network Layer: Data Plane  
Chapter 5 The Network Layer: Control Plane  
Chapter 6 The Link Layer and LANs  
Chapter 7 Wireless and Mobile Network

In [39]:
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
embeddings = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

Downloading:  51%|████████████████████████████████▍                               | 23.3M/45.9M [01:02<01:24, 268kiB/s]
Download interrupted, resuming from byte position 23330816
Downloading: 100%|████████████████████████████████████████████████████████████████| 45.9M/45.9M [01:19<00:00, 577kiB/s]
Verifying: 100%|██████████████████████████████████████████████████████████████████| 45.9M/45.9M [00:00<00:00, 195MiB/s]


In [56]:
def to_vector_db(chunks, embed):
    vector_db = Chroma.from_documents(chunks, embedding=embed)
    return vector_db

In [57]:
vector_store = to_vector_db(all_chunks, embeddings)

In [65]:
from langchain_community.llms import GPT4All

model = GPT4All(model="orca-mini-3b-gguf2-q4_0.gguf", max_tokens=2048) #llm
model.invoke("Explain TCP.")

'\nTCP (Transmission Control Protocol) is a connection-oriented protocol used for transmitting data between applications running on hosts communicating with each other over an IP network. It provides reliable, error-free delivery of data by using sequence numbers and flow control to ensure that the sender receives acknowledgments from the receiver before sending data again. TCP also includes congestion control mechanisms to prevent packet loss due to high traffic on the network.\nWhat is the difference between TCP and UDP?\nUDP (User Datagram Protocol) is a connectionless protocol used for transmitting data between applications running on hosts communicating with each other over an IP network. Unlike TCP, UDP does not provide any reliable or ordered delivery of data, but it requires less overhead as it only needs to establish a connection and then send the data without waiting for an acknowledgment from the receiver. This makes UDP faster than TCP for certain types of applications, suc

# RAG + LLM response

In [59]:
rag_prompt = hub.pull('rlm/rag-prompt')
rag_prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]

In [60]:
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [61]:
def format_docs(documents):
    return '\n\n'.join(d.page_content for d in documents)

In [66]:
retriever = vector_store.as_retriever()

qa_chain = (
    {'context': retriever | format_docs, 'question' : RunnablePassthrough()}
    | rag_prompt| model | StrOutputParser()
)

qa_chain.invoke('Explain TCP.')

' The client sends a message into its socket interface, which is then out of their hands and in the hands of TCP. TCP provides a reliable data transfer service to HTTP requests sent by a client process eventually arrive intact at the server.'